In [1]:
from sqlanalyzer import column_parser, unbundle
from sqlanalyzer.unbundle import *
import sqlparse
import re
import json
import pandas as pd
import time


def extract_subquery_fields(query, db_fields):
    formatter = column_parser.Parser(query)
    formatted = formatter.format_query(query)
    fields = formatter.match_queried_fields(formatted, db_fields)
    return fields


def compile_queried_cols(query_dict, df):
    all_cols = []
    for _,v in query_dict.items():
        if isinstance(v, dict):
            for _,v1 in v.items():
                all_cols.extend(extract_subquery_fields(v1, df))
        else:
            all_cols.extend(extract_subquery_fields(v, df))
    return all_cols

In [2]:
query = open('active_devs.sql').read()

formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
query_list = formatted_query.split('\n')

unbundled = unbundle.Unbundle(query)


In [3]:
# if is_cte(formatted_query):

cte_dict = formatter.parse_cte(formatted_query)
final_dict = {}
# for alias, query in cte_dict.items():
#     formatter = column_parser.Parser(query)
#     formatted_query = formatter.format_query(query)
#     try:
#         final_dict[alias] = unbundled.extract_query_dict(formatted_query)
#     except:
#         final_dict[alias] = formatted_query
    
#     with open('data.json', 'w') as outfile:
#         json.dump(final_dict, outfile)

# else:
#     final_dict = unbundled.extract_query_dict(query)
#     with open('data.json', 'w') as outfile:
#         json.dump(unbundled.extract_query_dict(query), outfile)


In [4]:
for k,v in cte_dict.items():
    print(k, '\n', v, '\n\n')

analytics_service_endpoint_mapping 
 SELECT b.*
   FROM
     (SELECT MAX(dt) AS dt
      FROM analytics.service_endpoint_mapping) a
   INNER JOIN analytics.service_endpoint_mapping b ON a.dt = b.dt),
 


web_dev 
 SELECT s.dt,
          COALESCE(em.platform, 'unknown') AS platform,
          COALESCE(service_org, 'other') AS service,
          account
   FROM sku.daily_by_account s
   INNER JOIN mapbox_customer_data.accounts a ON s.account = a.id
   AND a.dt = '{run_date}'
   LEFT JOIN analytics_service_endpoint_mapping em ON s.sku_id = em.sku_id
   AND em.in_sku IS NOT NULL
   AND em.parent_sku IS NULL
   WHERE s.dt BETWEEN DATE_SUB('{run_date}', 29) AND '{run_date}'
   GROUP BY 1,
            2,
            3,
            4),
 


mobile_dev 
 SELECT dt,
          'mobile' AS platform,
          CASE
              WHEN LOWER(useragent) RLIKE '(mapboxeventsnavigation|navigation-)' THEN 'navigation'
              WHEN LOWER(useragent) RLIKE '(mapboxeventsunity|mapbox-scenekit)' THEN 'un

In [5]:
cte_dict.keys()

dict_keys(['analytics_service_endpoint_mapping', 'web_dev', 'mobile_dev', 'studio_dev', 'web_mobile_studio', 'mau_cube', 'wau_cube', 'dau_cube', 'main'])

In [6]:
# for alias, query in cte_dict.items():
query = cte_dict['wau_cube']
alias = 'wau_cube'
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
try:
    final_dict[alias] = unbundled.extract_query_dict(formatted_query)
except:
    final_dict[alias] = formatted_query
    

In [7]:
final_dict

{'wau_cube': {'a': 'web_mobile_studio',
  "'studio'": "SELECT account,           CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service    FROM web_mobile_studio    WHERE dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'    GROUP BY account) b ON a.account = b.account AND b.service =",
  'main': "SELECT COALESCE(a.platform, 'other') AS platform,\n       COALESCE(a.service, 'unknown') AS service,\n       COUNT(DISTINCT a.account) AS num_devs,\n       COUNT(DISTINCT CASE\n                          WHEN b.account IS NOT NULL THEN NULL\n                          ELSE a.account\n                      END) AS num_devs_xstudio_only\nFROM web_mobile_studio a\nLEFT JOIN\nWHERE a.dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'\nGROUP BY COALESCE(a.platform, 'other'),\n         COALESCE(a.service, 'unknown') WITH CUBE), "}}

In [8]:
formatted_query

"SELECT COALESCE(a.platform, 'other') AS platform,\n       COALESCE(a.service, 'unknown') AS service,\n       COUNT(DISTINCT a.account) AS num_devs,\n       COUNT(DISTINCT CASE\n                          WHEN b.account IS NOT NULL THEN NULL\n                          ELSE a.account\n                      END) AS num_devs_xstudio_only\nFROM web_mobile_studio a\nLEFT JOIN\n  (SELECT account,\n          CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service\n   FROM web_mobile_studio\n   WHERE dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'\n   GROUP BY account) b ON a.account = b.account\nAND b.service = 'studio'\nWHERE a.dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'\nGROUP BY COALESCE(a.platform, 'other'),\n         COALESCE(a.service, 'unknown') WITH CUBE),"

In [11]:
# query_list_1 = formatted_query.split('\n')
# query_dict = {}

# sub_query = unbundled.delevel(query_list_0)
# query_dict = sub_query


In [10]:
query_dict

{}

In [12]:
query_1 = cte_dict['wau_cube']
formatter = column_parser.Parser(query_1)
formatted_query_1 = formatter.format_query(query_1)
query_list_1 = formatted_query_1.split('\n')


In [13]:
query_2 = cte_dict['web_dev']
formatter = column_parser.Parser(query_2)
formatted_query_2 = formatter.format_query(query_2)
query_list_2 = formatted_query_2.split('\n')


In [14]:
for i,l in enumerate(query_list_1): print(i, l)

0 SELECT COALESCE(a.platform, 'other') AS platform,
1        COALESCE(a.service, 'unknown') AS service,
2        COUNT(DISTINCT a.account) AS num_devs,
3        COUNT(DISTINCT CASE
4                           WHEN b.account IS NOT NULL THEN NULL
5                           ELSE a.account
6                       END) AS num_devs_xstudio_only
7 FROM web_mobile_studio a
8 LEFT JOIN
9   (SELECT account,
10           CONCAT_WS(',', COLLECT_SET(LOWER(service))) AS service
11    FROM web_mobile_studio
12    WHERE dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'
13    GROUP BY account) b ON a.account = b.account
14 AND b.service = 'studio'
15 WHERE a.dt BETWEEN DATE_SUB('{run_date}', 6) AND '{run_date}'
16 GROUP BY COALESCE(a.platform, 'other'),
17          COALESCE(a.service, 'unknown') WITH CUBE),


In [15]:
for i,l in enumerate(query_list_2): print(i, l)

0 SELECT s.dt,
1        COALESCE(em.platform, 'unknown') AS platform,
2        COALESCE(service_org, 'other') AS service,
3        account
4 FROM sku.daily_by_account s
5 INNER JOIN mapbox_customer_data.accounts a ON s.account = a.id
6 AND a.dt = '{run_date}'
7 LEFT JOIN analytics_service_endpoint_mapping em ON s.sku_id = em.sku_id
8 AND em.in_sku IS NOT NULL
9 AND em.parent_sku IS NULL
10 WHERE s.dt BETWEEN DATE_SUB('{run_date}', 29) AND '{run_date}'
11 GROUP BY 1,
12          2,
13          3,
14          4),


In [18]:
# _get_joins_pos
query_list = query_list_1

pos_delete, pos_where = [len(query_list)-1], len(query_list)
pos_join = []
for i, line in enumerate(query_list):
    if line.startswith('ORDER') or line.startswith('GROUP'):
        pos_delete.append(i)
    if line.startswith('FROM') and len(line.split(' ')) > 1:
        pos_join.append(i)
    elif line.startswith('FROM') and len(line.split(' ')) == 1:
        pos_join.append(i+1)
    if line.startswith('WHERE'):
        pos_where = i
    if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN') or line.startswith('RIGHT JOIN'):
        pos_join.append(i+1)

if min(pos_delete) == len(query_list)-1:
    pos_join.append(min(pos_delete))
else:
    pass

In [19]:
pos_join

[7, 9]

In [20]:
pos_where

15

In [24]:
# delevel

sub_query = {}
pos_join, pos_where = unbundled._get_joins_pos(query_list_0)
pos_join, pos_where


([4, 6, 8], 10)

In [ ]:
alias_pos = unbundled._get_alias_pos(query_list, pos_join, pos_where)
sub_query_pos = list(zip(pos_join, alias_pos))
sub_query, keep = unbundled._parse_sub_query(query_list, sub_query_pos)
main_query_pos = unbundled.main_query(query_list, sub_query_pos)
if main_query_pos != []:
    sub_query['main'] = '\n'.join([query_list[p] for p in main_query_pos])
sub_query['main'] = sub_query['main'] + ' ' + '\n'.join(keep)


In [ ]:
for alias, query in sub_query.items():

    formatter = column_parser.Parser(query)
    formatted_query = formatter.format_query(query)
    query_list = formatted_query.split('\n')

    if self.has_child(formatted_query) and alias != 'main':
        sub_query_dict = self.delevel(query_list)
        query_dict[alias] = sub_query_dict
        query_dict = clean_dict(query_dict)

    else:
        pass